# Avaliação Alternativa — Métodos Numéricos e Conversões

**Disciplina:** Matemática Computacional  
**Aluno:** Alessandro Reali Lopes Silva

### Este caderno reúne a resolução de três questões

1. Método da Bisseção com varredura de intervalos e verificação do sinal da derivada
2. Conversão de sequência binária para binário fracionário e para decimal, passo a passo
3. Conversão de número binário fracionário para decimal, retornando valor numérico e soma das parcelas.

In [60]:
from typing import Callable, Dict, Tuple, List, Any
from IPython.display import display, Markdown

def gerar_tabela_markdown(dados: List[Dict[str, Any]]) -> None:
    """Gera uma tabela em formato Markdown a partir de uma lista de dicionários."""
    if not dados:
        return "Nenhum dado disponível para exibir."
    
    tabela = ""

    chaves = dados[0].keys()

    tabela += f"| {' | '.join(chaves)} |\n"
    tabela += f"| {' | '.join(['---'] * len(chaves))} |\n"

    for dicionario in dados:
        tabela += f"| {' | '.join(str(dicionario[chave]) for chave in chaves)} |\n"
    
    display(Markdown(tabela))



# Questão 1 — Método da Bisseção

Resumo:
- Implementar o método da bisseção recebendo `f(x)` por parâmetro.
- Varredura de intervalos: iniciar em valores pequenos e crescer, detectando mudanças de sinal de `f`.
- Para cada intervalo com mudança de sinal, aplicar bisseção com um `ε` informado.
- Construir tabela de resultados (placeholder nesta seção; tabela final será adicionada quando recebida).
- Receber também `f'(x)` por parâmetro e verificar se a derivada muda de sinal em cada intervalo.

Objetivo: fornecer funções reutilizáveis e didáticas para localizar raízes e organizar os resultados.

In [61]:
def raiz_metodo_bissecao(
    funcao: Callable[[float], float],
    x_min: float,
    x_max: float,
    epsilon: float,
) -> Tuple[float, List[Dict[str, float]]]:
    """
    Encontra uma raiz da função no intervalo [x_min, x_max] usando o método da bisseção.
    Retorna a raiz encontrada e o histórico das iterações.
    """
    y_min = funcao(x_min)
    historico = []
    iteracao = 1

    while x_max - x_min > epsilon:
        meio = (x_min + x_max) / 2
        y_meio = funcao(meio)
        erro_atual = x_max - x_min

        historico.append({
            "iteracao": iteracao,
            "x_meio": meio,
            "f(x_meio)": y_meio,
            "erro": erro_atual
        })
        
        if y_meio == 0:
            return meio, historico
        
        elif y_min * y_meio < 0:
            x_max = meio
        else:
            x_min = meio
            y_min = y_meio

        iteracao += 1

    return (x_min + x_max) / 2, historico

def derivada_altera_sinal(
        derivada: Callable[[float], float],
        x_min: float,
        x_max: float,
) -> bool:
    """Verifica se a derivada muda de sinal no intervalo [x_min, x_max]."""
    return derivada(x_min) * derivada(x_max) < 0

def varredura_raizes(
    funcao: Callable[[float], float],
    derivada: Callable[[float], float],
    inicio: float,
    fim: float,
    passo: float,
    epsilon: float,
) -> List[Dict[str, Any]]:
    """Varre o intervalo [inicio, fim] em passos de 'passo' para encontrar raízes da função."""
    x_atual = inicio
    y_atual = funcao(x_atual)
    resultados = []

    while x_atual < fim:
        x_proximo = min(x_atual + passo, fim)
        y_proximo = funcao(x_proximo)

        # Verifica se a raiz está exatamente no ponto atual
        if y_atual == 0:
            resultados.append({
                "raiz": x_atual,
                "intervalo": (x_atual, x_atual),
                "derivada_muda_sinal": False,
                "historico": []
            })
        
        # Verifica se há mudança de sinal entre os pontos atual e próximo
        elif y_atual * y_proximo < 0:
            raiz, historico = raiz_metodo_bissecao(funcao, x_atual, x_proximo, epsilon)
            resultados.append({
                "raiz": raiz,
                "intervalo": (x_atual, x_proximo),
                "derivada_muda_sinal": derivada_altera_sinal(derivada, x_atual, x_proximo),
                "historico": historico
            })
        
        x_atual = x_proximo
        y_atual = y_proximo
    
    # Verifica se a raiz está exatamente no ponto final
    if y_atual == 0:
        resultados.append({
            "raiz": x_atual,
            "intervalo": (x_atual, x_atual),
            "derivada_muda_sinal": False,
            "historico": []
        })

    return resultados

In [62]:
"""Célula de execução do método da bisseção para encontrar raízes de uma função em um intervalo dado (questão 1)"""

def funcao_exemplo(x: float) -> float:
    """
    função exemplo
    pode ser alterada para testar outras funções
    """
    return x**3 - x - 2

def derivada_exemplo(x: float) -> float:
    """
    derivada da função exemplo
    pode ser alterada para testar outras funções
    lembre-se de que a derivada deve corresponder à função usada
    """
    return 3 * x**2 - 1

inicio = 0.0
fim = 5.0
passo = 0.5
epsilon = 10 ** (-3)

resultados = varredura_raizes(
    funcao=funcao_exemplo,
    derivada=derivada_exemplo,
    inicio=inicio,
    fim=fim,
    passo=passo,
    epsilon=epsilon
)

for i, resultado in enumerate(resultados, start=1):
    display(Markdown(f"""
### Raiz Encontrada #{i}
- Intervalo: {list(map(lambda x: round(x, 3), resultado['intervalo']))}
- Raiz Aproximada: {round(resultado['raiz'], 6)}
- Derivada Muda de Sinal: {"Sim" if resultado['derivada_muda_sinal'] else "Não"}
"""))

    if resultado['historico']:
        display(Markdown("#### Histórico das Iterações:"))
        gerar_tabela_markdown(resultado['historico'])
    else:
        display(Markdown("Nenhum histórico de iterações disponível (raiz exata encontrada)."))


### Raiz Encontrada #1
- Intervalo: [1.5, 2.0]
- Raiz Aproximada: 1.520996
- Derivada Muda de Sinal: Não


#### Histórico das Iterações:

| iteracao | x_meio | f(x_meio) | erro |
| --- | --- | --- | --- |
| 1 | 1.75 | 1.609375 | 0.5 |
| 2 | 1.625 | 0.666015625 | 0.25 |
| 3 | 1.5625 | 0.252197265625 | 0.125 |
| 4 | 1.53125 | 0.059112548828125 | 0.0625 |
| 5 | 1.515625 | -0.034053802490234375 | 0.03125 |
| 6 | 1.5234375 | 0.012250423431396484 | 0.015625 |
| 7 | 1.51953125 | -0.010971248149871826 | 0.0078125 |
| 8 | 1.521484375 | 0.0006221756339073181 | 0.00390625 |
| 9 | 1.5205078125 | -0.005178886465728283 | 0.001953125 |


# Questão 2 — Sequência binária → binário fracionário → decimal

Resumo:
- Receber uma sequência binária (lista de bits ou string) e construir o número binário fracionário correspondente (0.xxxx).
- Converter esse binário fracionário para o valor decimal, passo a passo (didático), sem estruturas prontas.
- Mostrar as parcelas (bit × 2^{-posição}) e o somatório final.

Objetivo: reforçar a compreensão de posições fracionárias em base 2.

In [63]:
def separar_componentes_binario_32bits(binario: str) -> Tuple[str, str, str]:
    """Separa uma string binária de 32 bits em sinal, expoente e mantissa."""
    if len(binario) != 32:
        raise ValueError(f"A string binária '{binario}' deve ter exatamente 32 bits.")
    
    # Bit de sinal 0 (positivo) ou 1 (negativo)
    sinal = binario[0]
    # Bits do expoente (8 bits)
    expoente = binario[1:9]
    # Bits da mantissa (23 bits)
    mantissa = binario[9:]
    
    return sinal, expoente, mantissa

def binario_para_inteiro(binario: str) -> int:
    """Converte uma string binária para um inteiro decimal."""
    valor = 0
    for i, bit in enumerate(reversed(binario)):
        if bit == '1':
            valor += 2 ** i

    return valor

def fracao_binaria_para_decimal(fracao_binaria: str) -> float:
    """Converte a parte fracionária de uma string binária para decimal."""
    valor = 0.0
    for i, bit in enumerate(fracao_binaria, start=1):
        if bit == '1':
            valor += 2 ** -i

    return valor

def converter_binario_32bits_para_float(binario: str, bias: int) -> float:
    """Converte uma string binária de 32 bits para um número de ponto flutuante decimal."""
    try:
        sinal, expoente_bin, mantissa_bin = separar_componentes_binario_32bits(binario)
    except ValueError as e:
        print(e)
        return 0.0
    
    sinal_decimal = int(sinal)
    expoente_decimal = binario_para_inteiro(expoente_bin)
    mantissa_decimal = fracao_binaria_para_decimal(mantissa_bin)

    # Formula do IEEE 754 para ponto flutuante
    # Valor = (-1)^sinal * (1 + mantissa) * 2^(expoente - bias)
    valor_float = ((-1) ** int(sinal_decimal)) * (1 + mantissa_decimal) * (2 ** (expoente_decimal - bias))
    
    return valor_float
    

In [64]:
"""Célula de execução da conversão de binário para float (questão 2)"""

bias = 127  # Bias para ponto flutuante de 32 bits (IEEE 754)
binarios_teste = [
    "00111110010100000000000000000000",
    "01000001011100000000000000000000",
    "11000000101010000000000000000000",
    "00111111100000000000000000000000",
]

tabela = """
| Binário (32 bits) | Sinal | Expoente | Mantissa |Decimal (Float) |
| --- | :-: | :-: | :-: | :-: |
"""

for binario in binarios_teste:
    sinal = binario[0]
    expoente = binario[1:9]
    mantissa = binario[9:]
    valor = converter_binario_32bits_para_float(binario, bias)
    tabela += f"| {binario} | {sinal} | {expoente} | {mantissa} | {valor} |\n"

display(Markdown(tabela))


| Binário (32 bits) | Sinal | Expoente | Mantissa |Decimal (Float) |
| --- | :-: | :-: | :-: | :-: |
| 00111110010100000000000000000000 | 0 | 01111100 | 10100000000000000000000 | 0.203125 |
| 01000001011100000000000000000000 | 0 | 10000010 | 11100000000000000000000 | 15.0 |
| 11000000101010000000000000000000 | 1 | 10000001 | 01010000000000000000000 | -5.25 |
| 00111111100000000000000000000000 | 0 | 01111111 | 00000000000000000000000 | 1.0 |


# Questão 3 — Binário fracionário → decimal

Resumo:
- Receber um número binário com parte inteira e fracionária (ex.: "101.011") e converter para decimal.
- Implementação passo a passo: somar contribuições da parte inteira (potências de 2 não negativas) e da parte fracionária (potências negativas).
- Retornar valor numérico (float) e também as somas separadas (inteira, fracionária) para conferência.
- Integração futura com script a ser fornecido.

Objetivo: consolidar a conversão completa de base 2 para base 10.

In [65]:
def converter_binario_fixo(binario: str) -> float:
    """Converte uma string binária representando um número em ponto fixo para decimal."""
    binario = binario.replace(',', '.')
    
    if '.' in binario:
        parte_inteira, parte_fracionaria = binario.split('.')
    else:
        parte_inteira, parte_fracionaria = binario, ''
    
    valor_inteiro = binario_para_inteiro(parte_inteira)
    valor_fracionario = fracao_binaria_para_decimal(parte_fracionaria)
    
    return valor_inteiro + valor_fracionario

def somar_binarios_fixo(binarios: List[str]) -> float:
    """Soma uma lista de strings binárias representando números em ponto fixo."""
    soma = 0.0
    for binario in binarios:
        soma += converter_binario_fixo(binario)

    return soma


In [66]:
"""Célula para conversão de binário de ponto fixo para decimal (questão 3)"""

lista_binarios = [
    "1101,101",
    "1010,11",
    "0110,01",
]

tabela = """
| Binário (Ponto Fixo) | Decimal |
| --- | :-: |
"""

for binario in lista_binarios:
    valor = converter_binario_fixo(binario)
    tabela += f"| {binario} | {valor} |\n"

display(Markdown(tabela))
display(Markdown(f"**Soma Total:** {somar_binarios_fixo(lista_binarios):.6f}"))


| Binário (Ponto Fixo) | Decimal |
| --- | :-: |
| 1101,101 | 13.625 |
| 1010,11 | 10.75 |
| 0110,01 | 6.25 |


**Soma Total:** 30.625000